# Minimizing Shipping Costs in a Regional Supply Chain 

## Problem Description

A tech manufacturer in the Bay Area operates two factories in San Jose and Oakland, supplying products to six major customers across the region. To efficiently distribute products, the network includes four strategically located depots in Palo Alto, Fremont, San Francisco, and Santa Cruz. Each depot has a maximum throughput, and each factory has a production capacity limit. Shipping products incurs costs that vary depending on the source and destination. 


Our supply network has two factories, in San Jose and Oakland, that produce a product. Each has a maximum production capacity:

| Factory | Supply (tons) |
| --- | --- |
| San Jose | 160,000 |
| Oakland |  210,000 |

Factory output is routed through four depots, each limited by throughput capacity. Depots act as intermediaries, they do not produce or consume goods but serve to efficiently channel products to customers.

| Depot | Throughput (tons) |
| --- | --- |
| Palo Alto | 75,000 |
| Fremont | 55,000 |
| San Francisco | 110,000 |
| Santa Cruz | 45,000 |

Our network has six customers, each with a given demand.

| Customer | Demand (tons) |
| --- | --- |
| C1 | 60,000 |
| C2 | 15,000 |
| C3 | 45,000 |
| C4 | 30,000 |
| C5 | 65,000 |
| C6 | 25,000 |

Shipping costs (in dollars per ton) are provided in the table below. Columns represent source location and rows represent destinations. For example, shipping from San Jose to SF costs $1.20 per ton. A dash (–) indicates that a shipment along that route is not possible (e.g., Oakland cannot ship directly to Palo Alto).


| To   | San Jose | Oakland | Palo Alto | Fremont | San Francisco | Santa Cruz |
|--------------|-----------|----------|------------|-----------|----------------|-------------|
| **Depots**   |           |          |            |           |                |             |
| Palo Alto    | 0.6       | -        |            |           |                |             |
| Fremont      | 0.6       | 0.36     |            |           |                |             |
| SF           | 1.2       | 0.6      |            |           |                |             |
| Santa Cruz   | 0.24      | 0.24     |            |           |                |             |
| **Customers**|           |          |            |           |                |             |
| C1           | 1.2       | 2.4      | -          | 1.2       | -              | -           |
| C2           | -         | -        | 1.8        | 0.6       | 1.8            | -           |
| C3           | 1.8       | -        | 0.6        | 0.6       | 2.4            | 0.24        |
| C4           | 2.4       | -        | 1.8        | 1.2       | -              | 1.8         |
| C5           | -         | -        | -          | 0.6       | 0.6            | 0.6         |
| C6           | 1.2       | -        | 1.2        | -         | 1.8            | 1.8         |

---
## Model Formulation

### Set & Indices

$f \in \text{Factories}=\{\text{San Jose}, \text{Oakland}\}$

$d \in \text{Depots}=\{\text{Palo Alto}, \text{Fremont}, \text{SF}, \text{Santa Cruz}\}$

$c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}$

### Parameters

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from factory $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for goods at customer $c$ (in tons).

### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.

### Objective Function

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}}
\end{equation}

### Constraints

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}

- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity**: Flow into a depot must respect depot capacity.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d}
\quad \forall d \in \text{Depots}
\end{equation}



In [6]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 7.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [8]:
# Dictionary to capture factory supply limits, depot throughput limits, and customer demand

supply = dict({'San Jose': 160000,
               'Oakland': 210000})

through = dict({'Palo Alto': 75000,
                'Fremont': 55000,
                'San Francisco': 110000,
                'Santa Cruz': 45000})

demand = dict({'CU1': 60000,
               'CU2': 15000,
               'CU3': 45000,
               'CU4': 30000,
               'CU5': 65000,
               'CU6': 25000})

# Dictionary to capture shipping costs.

arcs, cost = gp.multidict({
    ('San Jose', 'Palo Alto'): 0.6,
    ('San Jose', 'Fremont'): 0.6,
    ('San Jose', 'San Francisco'): 1.2,
    ('San Jose', 'Santa Cruz'): 0.24,
    ('San Jose', 'CU1'): 1.2,
    ('San Jose', 'CU3'): 1.8,
    ('San Jose', 'CU4'): 2.4,
    ('San Jose', 'CU6'): 1.2,
    ('Oakland', 'Fremont'): 0.36,
    ('Oakland', 'San Francisco'): 0.6,
    ('Oakland', 'Santa Cruz'): 0.24,
    ('Oakland', 'CU1'): 2.4,
    ('Palo Alto', 'CU2'): 1.8,
    ('Palo Alto', 'CU3'): 0.6,
    ('Palo Alto', 'CU5'): 1.8,
    ('Palo Alto', 'CU6'): 1.2,
    ('Fremont', 'CU1'): 1.2,
    ('Fremont', 'CU2'): 0.6,
    ('Fremont', 'CU3'): 0.6,
    ('Fremont', 'CU4'): 1.2,
    ('Fremont', 'CU5'): 0.6,
    ('San Francisco', 'CU2'): 1.8,
    ('San Francisco', 'CU3'): 2.4,
    ('San Francisco', 'CU5'): 0.6,
    ('San Francisco', 'CU6'): 1.8,
    ('Santa Cruz', 'CU3'): 0.24,
    ('Santa Cruz', 'CU4'): 1.8,
    ('Santa Cruz', 'CU5'): 0.6,
    ('Santa Cruz', 'CU6'): 1.8
})


## Model Deployment

We build an optimization model to simulate how products move through the network. Each variable tracks the volume shipped between connected locations. Because shipping costs are already tied to these flows, the model automatically minimizes total transportation cost.


In [9]:
model = gp.Model('SupplyNetworkDesign')
flow = model.addVars(arcs, obj=cost, name="flow")

Restricted license - for non-production use only - expires 2026-11-23


Our first constraints require the total flow along arcs leaving a factory to be at most as large as the supply capacity of that factory.

In [10]:
# Production capacity limits

factories = supply.keys()
factory_flow = model.addConstrs((gp.quicksum(flow.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory")

Our next constraints require the total flow along arcs entering a customer to be equal to the demand from that customer.

In [11]:
# Customer demand

customers = demand.keys()
customer_flow = model.addConstrs((gp.quicksum(flow.select('*', customer)) == demand[customer]
                                  for customer in customers), name="customer")

Our final constraints relate to depots.  The first constraints require that the total amount of product entering the depot must equal the total amount leaving.

In [12]:
# Depot flow conservation

depots = through.keys()
depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

The second set limits the product passing through the depot to be at most equal the throughput of that deport.

In [13]:
# Depot throughput

depot_capacity = model.addConstrs((gp.quicksum(flow.select('*', depot)) <= through[depot]
                                   for depot in depots), name="depot_capacity")

In [15]:
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 24.6.0 24G90)

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0x356efb52
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9680000e+05   1.937500e+04   0.000000e+00      0s
       8    2.6040000e+05   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.604000000e+05


---
## Analysis

Product demand from all of our customers can be satisfied for a total cost of $\$260,040$. The optimal plan is as follows.

In [18]:
rows = []
for arc in arcs:
    if flow[arc].x > 1e-6:
        rows.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x})

product_flow = pd.DataFrame(rows, columns=["From", "To", "Flow"])

# optional: set blank index like you had before
if len(product_flow):
    product_flow.index = [''] * len(product_flow)

product_flow


,From,To,Flow
,San Jose,CU1,60000.0
,San Jose,CU6,25000.0
,Oakland,Fremont,55000.0
,Oakland,San Francisco,55000.0
,Oakland,Santa Cruz,45000.0
,Fremont,CU2,15000.0
,Fremont,CU4,30000.0
,Fremont,CU5,10000.0
,San Francisco,CU5,55000.0
,Santa Cruz,CU3,45000.0
